In [33]:
import numpy as np;
import cv2 as cv;
from os import path

def normalize_filter(filter_array):
    return filter_array/np.sum(np.abs(filter_array))


In [34]:
# Filter 1 (Filter A) - Laplacian of Gaussian (LoG) Filter
filter1 = normalize_filter(np.array([
    [0, -1, -1, -1, 0],
    [-1, 2, 2, 2, -1],
    [-1, 2, 8, 2, -1],
    [-1, 2, 2, 2, -1],
    [0, -1, -1, -1, 0]
]))

# Filter 2 (Filter B) - Gaussian Blur Filter
filter2 = normalize_filter(np.array([
    [1, 4, 6, 4, 1],
    [4, 16, 24, 16, 4],
    [6, 24, 36, 24, 6],
    [4, 16, 24, 16, 4],
    [1, 4, 6, 4, 1]
]))

# Filter 3 (Filter C) - Uniform Box Filter
filter3 = normalize_filter(np.array([
    [5, 5, 5, 5, 5],
    [5, 5, 5, 5, 5],
    [5, 5, 5, 5, 5],
    [5, 5, 5, 5, 5],
    [5, 5, 5, 5, 5]
]))

# Filter 4 (Filter D) - Sharpening Filter (High-Boost Filter)
filter4 = normalize_filter(np.array([
    [0, -1, -1, -1, 0],
    [-1, 2, 2, 2, -1],
    [-1, 2, 16, 2, -1],
    [-1, 2, 2, 2, -1],
    [0, -1, -1, -1, 0]
]))


In [35]:

#  Reading the image using OpenCV. Handled file not found situation.

img = cv.imread('./road98.png')
if img is None:
    print("Error: Image not found.")
    exit()
print(f"Image converted to type : {type(img)}" )


Image converted to type : <class 'numpy.ndarray'>


In [36]:
#  Converting the 3D Numpy Array containing BGR Image to a 2D array containing the Grayscale Image.

gray_img = np.round(0.114 * img[:, :, 0] + 0.587 * img[:, :, 1] + 0.299 * img[:, :, 2]).astype(np.uint8)

In [37]:
max_val= np.max(gray_img)
min_val = np.min(gray_img)

original_img = np.round(255*((gray_img-min_val)/(max_val-min_val))).astype(np.uint8);
cv.imwrite('./original.jpg',original_img ) # Saving as an image.

True

In [38]:
# Adding padding.
filter_offset = filter1.shape[0]//2  # Offset for the filter.
print(filter_offset)
for i in range(filter_offset) :
    if i==0:
        padded_img = np.insert(original_img, 0, 0, axis=0)
    else: 
         padded_img = np.insert(padded_img, 0, 0, axis=0)
    padded_img = np.insert(padded_img[::-1],0 , 0, axis=0)
    padded_img = np.insert(padded_img, 0, 0, axis=1)
    padded_img = np.insert(padded_img, padded_img.shape[1], 0, axis=1)
print(padded_img)
# print()
print(original_img.shape)
print(padded_img.shape)

2
[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0 221 ... 228   0   0]
 ...
 [  0   0 221 ... 180   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
(267, 400)
(271, 404)


In [39]:
def apply_filter_and_save_image(img_filter, img_name):
    new_img = np.zeros_like(original_img)
    for i in range(original_img.shape[0]):
        for j in range(original_img.shape[1]):
            new_img[i][j] = np.sum(padded_img[i:i+img_filter.shape[0], j:j+img_filter.shape[1]] * img_filter)
    print(new_img)
    cv.imwrite('./'+img_name,new_img ) # Saving as an image.

In [40]:
apply_filter_and_save_image(filter1, 'filter1.jpg')
apply_filter_and_save_image(filter2, 'filter2.jpg')
apply_filter_and_save_image(filter3, 'filter3.jpg')
apply_filter_and_save_image(filter4, 'filter4.jpg')

[[ 61  80  69 ...  71  82  63]
 [ 79 110  93 ...  97 115  83]
 [ 66  91  73 ...  79  97  71]
 ...
 [ 66  90  71 ...  56  70  51]
 [ 78 108  89 ...  73  88  63]
 [ 61  79  66 ...  55  65  50]]
[[104 143 155 ... 160 149 108]
 [142 195 210 ... 219 204 149]
 [150 206 222 ... 236 220 160]
 ...
 [149 202 214 ... 171 160 117]
 [140 190 201 ... 163 153 112]
 [103 140 148 ... 121 114  83]]
[[ 79 107 134 ... 140 112  83]
 [106 142 178 ... 187 150 112]
 [132 177 222 ... 235 188 140]
 ...
 [130 172 214 ... 172 137 103]
 [104 138 171 ... 138 110  83]
 [ 78 103 128 ... 105  84  63]]
[[ 90 106  98 ... 100 109  93]
 [105 131 117 ... 122 136 109]
 [ 94 114 100 ... 107 122 101]
 ...
 [ 93 113  96 ...  76  88  73]
 [103 128 112 ...  91 104  84]
 [ 90 105  94 ...  77  86  73]]
